In [ ]:
import geopandas as gpd
import pandas as pd
import json
import plotly.express as px
import zipfile
from urllib import request
import numpy as np


In [ ]:
response = request.urlopen('https://services1.arcgis.com/Nifc7wlHaBPig3Q3/arcgis/rest/services/COVID_PA_Counties/FeatureServer/0/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&resultOffset=0&resultRecordCount=68&resultType=standard&cacheHint=true')
dat = response.read()
data = json.loads(dat)
df = pd.json_normalize(data['features'])
df.columns=df.columns.str.replace("attributes.",'',regex=False).str.lower()
df=df[['county','cases_per_100000']][df.cases_per_100000.notna()]
df['county']=df.county.str.title()
map1=gpd.read_file("https://data.pa.gov/api/geospatial/2axz-wupj?method=export&format=GeoJSON")
map1['county']=map1.county_nam.str.title()
map1=map1[['county','geometry']]
map1.to_file("map1.geojson",driver='GeoJSON')
with open ("map1.geojson",'r') as infile:
    counties = json.load(infile)
fig = px.choropleth_mapbox(df,geojson=counties,
                          locations='county',featureidkey='properties.county',
                          color='cases_per_100000',
                          hover_name='county',
                          range_color=(df.cases_per_100000.describe()[1]-df.cases_per_100000.describe()[2],df.cases_per_100000.describe()[1]+df.cases_per_100000.describe()[2]),
                          mapbox_style='carto-positron',
                          color_continuous_scale='viridis',
                          zoom=6.3,center = {"lat": 40.9998307, "lon": -77.6162591},
                          opacity=0.5)
fig.show()
#df.to_csv("county_covid_19_"+str(today.month)+"_"+str(today.day)+".csv",index=None)

In [ ]:
response = request.urlopen("https://services1.arcgis.com/Nifc7wlHaBPig3Q3/arcgis/rest/services/Covid_Cases_County/FeatureServer/0/query?f=json&where=county%3D%27Bucks%27&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=ObjectId%2Ccases%2Cdate&orderByFields=date%20asc&resultOffset=0&resultRecordCount=32000&resultType=standard&cacheHint=true")
dat = response.read()
data = json.loads(dat)
bucks_cases = pd.json_normalize(data['features'])
bucks_cases.columns=bucks_cases.columns.str.replace("attributes.",'',regex=False).str.lower()
bucks_cases['date']=pd.to_datetime(bucks_cases['date'],unit="ms",origin='unix')
bucks_cases['date']=bucks_cases.date.dt.date
bucks_cases['case_diff_av7d']=(bucks_cases.cases.rolling(window=7).mean().round(0))
bucks_cases['county']='Bucks'
response = request.urlopen("https://services1.arcgis.com/Nifc7wlHaBPig3Q3/arcgis/rest/services/Covid_Cases_County/FeatureServer/0/query?f=json&where=county%3D%27Montgomery%27&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=ObjectId%2Ccases%2Cdate&orderByFields=date%20asc&resultOffset=0&resultRecordCount=32000&resultType=standard&cacheHint=true")
dat = response.read()
data = json.loads(dat)
#today=pd.to_datetime(date.today())
montgomery_cases = pd.json_normalize(data['features'])
montgomery_cases.columns=montgomery_cases.columns.str.replace("attributes.",'',regex=False).str.lower()
montgomery_cases['date']=pd.to_datetime(montgomery_cases['date'],unit="ms",origin='unix')
montgomery_cases['date']=montgomery_cases.date.dt.date
montgomery_cases['case_diff_av7d']=(montgomery_cases.cases.rolling(window=7).mean().round(0))
montgomery_cases['county']='Montgomery'
response = request.urlopen("https://services1.arcgis.com/Nifc7wlHaBPig3Q3/arcgis/rest/services/Covid_Cases_County/FeatureServer/0/query?f=json&where=county%3D%27Allegheny%27&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=ObjectId%2Ccases%2Cdate&orderByFields=date%20asc&resultOffset=0&resultRecordCount=32000&resultType=standard&cacheHint=true")
dat = response.read()
data = json.loads(dat)
df1=pd.concat([bucks_cases,montgomery_cases,])
px.line(df1.replace(np.nan,0),y="case_diff_av7d",x='date',color='county',title='7-Day Average new cases in Bucks and Montgomery counties')